In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
import pickle
import os

### Useful Functions

In [9]:
def list_pickles(path):
    pickle_list = []
    for name in os.listdir(path):
        if name[-7:]=='.pickle':
            pickle_list.append(name)
    return pickle_list


def count_particles(shower_summary,central_det=True):
    diccionario=shower_summary[0]
    df1=shower_summary[1]
    df2=shower_summary[2]
    df3=shower_summary[3]
    energy=diccionario["PRME"]
    A1=len(df1)
    A2=len(df2)
    A3=len(df3)
    part_count= A1+A2+A3
    if central_det:
        df0=shower_summary[4]
        A0=len(df0)
        part_count=part_count + A0
    return energy,part_count

## Particle Count Histogram

### Data loading

In [ ]:
parent_dir='C:\\Users\\cg_h2\\Documents\\pucp_array\\pickles\\'

showers=[]
pickle_names=list_pickles(parent_dir)
for pickle_name in pickle_names:
    pickle_path=os.path.join(parent_dir,pickle_name)
    with open(pickle_path, 'rb') as file:
        showers_i = pickle.load(file)
    showers=showers+showers_i

## Loaded data can be found in list: showers


### Data Processing

In [ ]:

e_list=[]
count_list=[]
for shower in showers:
    energy,part_count= count_particles(shower,central_det=True)
    e_list.append[energy]
    count_list.append[part_count]

e_unique=list(set(e_list))
energies_and_counts={}
for energy in e_unique:
    c_list=[]
    for e,c in zip(e_list,count_list):
        if e==energy:
            c_list.append(c)
    energies_and_counts[energy]=c_list
    
    
#in the end, we have:  e_unique (unique values of the energies)
#                      energies_and_counts (dictionary that contains one list per energy, containing particle counts for each shower)

### Plot

In [ ]:
plt.figure(figsize=(8, 6))
for energy in e_unique:
    data=energies_and_counts[energy]
    plt.hist(data, bins=15, label=f'PRME: {energy}')

plt.xlabel('Particle count')
plt.ylabel('Multiplicity')
plt.title('Total particle counts for different energies')
plt.legend()
plt.grid(True)
plt.show()
